In [1]:
import sys
sys.path.append('/home/UR/kontaous/Bureau/Stage/')
from Analyses.traitement import *
import pandas as pd
import numpy as np
from Analyses.traitement import *


In [2]:
tracking = pd.read_csv("../../../Tracking_data/MontPellier(2)_(2)HAC.csv")
events = pd.read_csv("../../Data/match_data/match_Montpellier.csv")
tracking, events = prepare_data(tracking, events)
tracking = tracking.sort_values(by="system_time")
tracking.set_index('frame', inplace=True)
#tracking.rename(columns={'player_name': 'player.name', 'team': 'team.name', 'x': 'location.x', 'y': 'location.y', }, inplace=True)
tracking.drop(['system_time', 'birthdate','x','y', 'Gx', 'Gy', 'id', 'num', 'player', 'Id_tracking', 'MatchId'], axis=1, inplace=True)
tracking.rename(columns={'player_name': 'player.name', 'team': 'team.name'}, inplace=True)
#data_tracking = "../../../../Tracking_data/new_MontPellier.csv"
#sauvegarder_dataframe_en_csv(tracking, data_tracking)
tracking = tracking[tracking['team.name'] == "HAC"]


In [3]:
df_positions = tracking[:11]
df_positions

,datetime,period,team.name,player.name,role,In,Out
frame,,,,,,,
0,2023-08-13 15:00:00,1,HAC,Loic Nego,Défenseur,1691938800000.0,1691943865720.0
0,2023-08-13 15:00:00,1,HAC,Oussama Targhalline,Milieu de terrain,1691938800000.0,0.0
0,2023-08-13 15:00:00,1,HAC,Arouna Sangante,Défenseur,1691938800000.0,0.0
0,2023-08-13 15:00:00,1,HAC,Issa Soumare,Attaquant,1691938800000.0,1691943343880.0
0,2023-08-13 15:00:00,1,HAC,Rassoul Ndiaye,Milieu de terrain,1691938800000.0,1691943887320.0
0,2023-08-13 15:00:00,1,HAC,Josue Casimir,Attaquant,1691938800000.0,0.0
0,2023-08-13 15:00:00,1,HAC,Gautier Lloris,Défenseur,1691938800000.0,0.0
0,2023-08-13 15:00:00,1,HAC,Daler Kuzyaev,Milieu de terrain,1691938800000.0,1691944804720.0
0,2023-08-13 15:00:00,1,HAC,Yoann Salmier,Défenseur,1691938800000.0,0.0


In [4]:
df_passes = pd.read_csv('../../Data/Tracking/montpellier_passes.csv')
df_passes[df_passes['team.name'] == 'Le Havre']['player.name'].unique()


array(['Nabil Alioui', 'Oussama Targhalline', 'Daler Kuzyaev',
       'Arouna Sangante', 'Josue Casimir', 'Yoann Salmier', 'Loïc Négo',
       'Arthur Desmas', "Rassoul N'Diaye", 'Gautier Lloris',
       'Issa Soumaré', 'Andy Élysée Logbo', 'Antoine Joujou',
       'Samuel Grandsir', 'Yassine Kechta', 'Abdoulaye Touré'],
      dtype=object)

In [5]:
modifications = {
    "Rassoul N'Diaye": 'Rassoul Ndiaye',
    'Loïc Négo': 'Loic Nego',
    'Issa Soumaré' : 'Issa Soumare'
}

df_passes.replace({'player.name': modifications, 'pass.recipient.name': modifications}, inplace=True)
df_passes

,datetime,team.name,period,player.name,pass.recipient.name,pass.length
0,2023-08-13 00:00:00.474,Montpellier,1,Téji Savanier,Jordan Ferri,11.433723
1,2023-08-13 00:00:01.066,Le Havre,2,Nabil Alioui,Oussama Targhalline,11.815245
2,2023-08-13 00:00:02.175,Montpellier,1,Jordan Ferri,Falaye Sacko,22.375880
3,2023-08-13 00:00:02.815,Le Havre,2,Oussama Targhalline,Issa Soumare,52.500000
4,2023-08-13 00:00:05.783,Montpellier,1,Falaye Sacko,Christopher Jullien,15.508062
...,...,...,...,...,...,...
906,2023-08-13 00:51:58.203,Le Havre,1,Arthur Desmas,Issa Soumare,57.215470
907,2023-08-13 00:52:01.596,Montpellier,2,Christopher Jullien,Falaye Sacko,16.046183
908,2023-08-13 00:52:01.675,Montpellier,1,Christopher Jullien,Akor Jerome Adams,26.931395
909,2023-08-13 00:52:40.510,Montpellier,1,Jordan Ferri,Wahbi Khazri,9.948870


In [6]:
import plotly.graph_objs as go

""" on va faire un dictionnaire des positions en fonction des rôles
pour placer les joueurs sur le terrain """

role_positions = {
    "Gardien": [(3, 40)],
    "Défenseur": [(12, 30), (12, 50), (25, 10), (25, 70)],
    "Milieu de terrain": [(50, 40), (80, 20), (80, 60)],
    "Attaquant": [(100, 10), (100, 70), (110, 40)]
}


def get_position(role, index, role_positions, role_count):
    if role in role_positions and index < len(role_positions[role]):
        return role_positions[role][index]
    else:
        if role == "Défenseur":
            role = "Milieu de terrain"
        elif role == "Milieu de terrain":
            role = "Attaquant"
        elif role == "Attaquant":
            role = "Défenseur"
        return get_position(role, role_count[role], role_positions, role_count)

def visualize_pass_network(player_positions_df, passes_df):
    role_count = {"Défenseur": 0, "Milieu de terrain": 0, "Attaquant": 0, "Gardien": 0}
    players_positions = {}
    
    # Boucler à travers chaque joueur pour définir sa position sur le terrain
    for index, row in player_positions_df.iterrows():
        role = row['role']
        position = get_position(role, role_count[role], role_positions, role_count)
        role_count[role] += 1
        players_positions[row['player.name']] = position

    # Calculer les informations des passes pour chaque joueur
    pass_info = passes_df.groupby(['player.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    pass_info_received = passes_df.groupby(['pass.recipient.name']).agg(
        num_passes_received=('pass.length', 'size'),
        avg_pass_length_received=('pass.length', 'mean')
    ).reset_index()

    # Créer des dictionnaires pour les infos de passes
    pass_info_dict = {
        row['player.name']: {
            'num_passes': row['num_passes'],
            'avg_pass_length': row['avg_pass_length']
        }
        for index, row in pass_info.iterrows()
    }

    pass_info_received_dict = {
        row['pass.recipient.name']: {
            'num_passes_received': row['num_passes_received'],
            'avg_pass_length_received': row['avg_pass_length_received']
        }
        for index, row in pass_info_received.iterrows()
    }

    # Tracer le terrain de football
    fig = go.Figure()

    # Ajouter des formes pour représenter le terrain de football
    terrain_shapes = [
        dict(type="rect", x0=0, y0=0, x1=120, y1=80, line=dict(color="Black")),  # Périmètre du terrain
        dict(type="line", x0=60, y0=0, x1=60, y1=80, line=dict(color="Black", dash="dot")),  # Ligne médiane
        dict(type="rect", x0=0, y0=18, x1=18, y1=62, line=dict(color="Black")),  # Surface de réparation
        dict(type="rect", x0=102, y0=18, x1=120, y1=62, line=dict(color="Black")),  # Surface de réparation opposée
        dict(type="rect", x0=0, y0=30, x1=5.5, y1=50, line=dict(color="Black")),  # Petit rectangle
        dict(type="rect", x0=114.5, y0=30, x1=120, y1=50, line=dict(color="Black")),  # Petit rectangle opposé
        dict(type="rect", x0=0, y0=36, x1=2, y1=44, line=dict(color="Black")),  # Zone du gardien
        dict(type="rect", x0=118, y0=36, x1=120, y1=44, line=dict(color="Black")),  # Zone du gardien opposée
        dict(type="circle", xref="x", yref="y", x0=50.4, y0=30.4, x1=69.6, y1=49.6, line_color="Black"),  # Cercle central
    ]
    fig.update_layout(shapes=terrain_shapes)

    # Ajouter les joueurs sur le terrain avec les infos de passes
    for player, pos in players_positions.items():
        passes_from_player = pass_info_dict.get(player, {'num_passes': 0, 'avg_pass_length': 0})
        passes_to_player = pass_info_received_dict.get(player, {'num_passes_received': 0, 'avg_pass_length_received': 0})

        # Texte affiché lorsqu'on survole un joueur
        hover_text = (
            f"Joueur: {player}<br>"
            f"Nombre de passes effectuées: {passes_from_player['num_passes']}<br>"
            f"Longueur moyenne des passes effectuées: {passes_from_player['avg_pass_length']:.2f} m<br>"
            f"Nombre de passes reçues: {passes_to_player['num_passes_received']}<br>"
            f"Longueur moyenne des passes reçues: {passes_to_player['avg_pass_length_received']:.2f} m"
        )

        fig.add_trace(go.Scatter(
            x=[pos[0]],
            y=[pos[1]],
            mode='markers+text',
            text=[player],
            textposition="top center",
            marker=dict(size=20, color='blue'),  # Taille du marqueur augmentée à 20
            hovertext=hover_text,
            hoverinfo="text"
        ))

    # Calculer le nombre et la longueur moyenne des passes entre les joueurs
    passes_between_players = passes_df.groupby(['player.name', 'pass.recipient.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    # Ajouter les passes
    for row in passes_between_players.itertuples():
        if row[1] in players_positions and row[2] in players_positions:
            start = players_positions[row[1]]
            end = players_positions[row[2]]
            # Texte affiché lorsqu'on survole une flèche de passe
            hover_text = (
                f"Passes de {row[1]} à {row[2]}<br>"
                f"Nombre de passes: {row.num_passes}<br>"
                f"Longueur moyenne des passes: {row.avg_pass_length:.2f} m"
            )
            fig.add_trace(go.Scatter(
                x=[start[0], end[0]],
                y=[start[1], end[1]],
                mode='lines',
                line=dict(width=1, color='red'),
                hovertext=hover_text,
                hoverinfo="text"
            ))

    # Inverser l'axe y pour que l'origine soit en bas à gauche
    fig.update_yaxes(autorange='reversed')

    # Configurer l'apparence générale du graphique
    fig.update_layout(
        title="Réseau de passes du Havre",
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False),
        width=1200,
        height=900
    )
    fig.show()

# Appel de la fonction pour créer la figure du réseau de passes
visualize_pass_network(df_positions, df_passes)


In [7]:
import plotly.graph_objs as go

# Dictionnaire des positions en fonction des rôles
role_positions = {
    "Gardien": [(3, 40)],
    "Défenseur": [(12, 30), (12, 50), (25, 10), (25, 70)],
    "Milieu de terrain": [(50, 40), (80, 20), (80, 60)],
    "Attaquant": [(100, 10), (100, 70), (110, 40)]
}

# Fonction pour obtenir la position en fonction du rôle et de l'index
def get_position(role, index, role_positions, role_count):
    if role in role_positions and index < len(role_positions[role]):
        return role_positions[role][index]
    else:
        # Gérer les excédents en attribuant des positions restantes
        if role == "Défenseur":
            role = "Milieu de terrain"
        elif role == "Milieu de terrain":
            role = "Attaquant"
        elif role == "Attaquant":
            role = "Défenseur"
        return get_position(role, role_count[role], role_positions, role_count)

# Fonction pour créer le graphique de passes
def visualize_pass_network(player_positions_df, passes_df):
    # Convertir les données de la dataframe en dictionnaires
    role_count = {"Défenseur": 0, "Milieu de terrain": 0, "Attaquant": 0, "Gardien": 0}
    players_positions = {}
    
    for index, row in player_positions_df.iterrows():
        role = row['role']
        position = get_position(role, role_count[role], role_positions, role_count)
        role_count[role] += 1
        players_positions[row['player.name']] = position

    # Calculer les informations des passes
    pass_info = passes_df.groupby(['player.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    pass_info_received = passes_df.groupby(['pass.recipient.name']).agg(
        num_passes_received=('pass.length', 'size'),
        avg_pass_length_received=('pass.length', 'mean')
    ).reset_index()

    # Créer des dictionnaires pour les infos de passes
    pass_info_dict = {
        row['player.name']: {
            'num_passes': row['num_passes'],
            'avg_pass_length': row['avg_pass_length']
        }
        for index, row in pass_info.iterrows()
    }

    pass_info_received_dict = {
        row['pass.recipient.name']: {
            'num_passes_received': row['num_passes_received'],
            'avg_pass_length_received': row['avg_pass_length_received']
        }
        for index, row in pass_info_received.iterrows()
    }

    # Tracer le terrain de football
    fig = go.Figure()

    # Ajouter des formes pour représenter le terrain de football
    terrain_shapes = [
        dict(type="rect", x0=0, y0=0, x1=120, y1=80, line=dict(color="Black")),
        dict(type="line", x0=60, y0=0, x1=60, y1=80, line=dict(color="Black", dash="dot")),
        dict(type="rect", x0=0, y0=18, x1=18, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=102, y0=18, x1=120, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=30, x1=5.5, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=114.5, y0=30, x1=120, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=36, x1=2, y1=44, line=dict(color="Black")),
        dict(type="rect", x0=118, y0=36, x1=120, y1=44, line=dict(color="Black")),
        dict(type="circle", xref="x", yref="y", x0=50.4, y0=30.4, x1=69.6, y1=49.6, line_color="Black"),
    ]
    fig.update_layout(shapes=terrain_shapes)

    # Ajouter les joueurs sur le terrain avec les infos de passes
    for player, pos in players_positions.items():
        passes_from_player = pass_info_dict.get(player, {'num_passes': 0, 'avg_pass_length': 0})
        passes_to_player = pass_info_received_dict.get(player, {'num_passes_received': 0, 'avg_pass_length_received': 0})

        # Calculer les passes entre ce joueur et les autres
        passes_to_other_players = passes_df[passes_df['player.name'] == player]
        passes_from_other_players = passes_df[passes_df['pass.recipient.name'] == player]

        passes_info_text = ""
        for idx, pass_row in passes_to_other_players.iterrows():
            recipient = pass_row['pass.recipient.name']
            if recipient in players_positions:
                passes_info_text += f"Passes vers {recipient}: {pass_row['pass.length']} m<br>"
        
        for idx, pass_row in passes_from_other_players.iterrows():
            sender = pass_row['player.name']
            if sender in players_positions:
                passes_info_text += f"Passes de {sender}: {pass_row['pass.length']} m<br>"

        hover_text = (
            f"Joueur: {player}<br>"
            f"Nombre de passes effectuées: {passes_from_player['num_passes']}<br>"
            f"Longueur moyenne des passes effectuées: {passes_from_player['avg_pass_length']:.2f} m<br>"
            f"Nombre de passes reçues: {passes_to_player['num_passes_received']}<br>"
            f"Longueur moyenne des passes reçues: {passes_to_player['avg_pass_length_received']:.2f} m<br>"
            f"{passes_info_text}"
        )

        fig.add_trace(go.Scatter(
            x=[pos[0]],
            y=[pos[1]],
            mode='markers+text',
            text=[player],
            textposition="top center",
            marker=dict(size=20, color='blue'),
            hovertext=hover_text,
            hoverinfo="text"
        ))

    # Calculer le nombre et la longueur moyenne des passes entre les joueurs
    passes_between_players = passes_df.groupby(['player.name', 'pass.recipient.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    # Ajouter les passes
    for row in passes_between_players.itertuples():
        if row[1] in players_positions and row[2] in players_positions:
            start = players_positions[row[1]]
            end = players_positions[row[2]]
            hover_text = (
                f"Passes de {row[1]} à {row[2]}<br>"
                f"Nombre de passes: {row.num_passes}<br>"
                f"Longueur moyenne des passes: {row.avg_pass_length:.2f} m"
            )
            fig.add_trace(go.Scatter(
                x=[start[0], end[0]],
                y=[start[1], end[1]],
                mode='lines',
                line=dict(width=2, color='red'),
                hovertext=hover_text,
                hoverinfo="text"
            ))

    fig.update_layout(
        title="Réseau de passes sur un terrain de football",
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False),
        width=1200,
        height=900
    )
    fig.show()


In [8]:
visualize_pass_network(df_positions, df_passes)

In [9]:
import plotly.graph_objs as go

# Dictionnaire des positions en fonction des rôles
role_positions = {
    "Gardien": [(3, 40)],
    "Défenseur": [(12, 30), (12, 50), (25, 10), (25, 70)],
    "Milieu de terrain": [(50, 40), (80, 20), (80, 60)],
    "Attaquant": [(100, 10), (100, 70), (110, 40)]
}

# Fonction pour obtenir la position en fonction du rôle et de l'index
def get_position(role, index, role_positions, role_count):
    if role in role_positions and index < len(role_positions[role]):
        return role_positions[role][index]
    else:
        # Gérer les excédents en attribuant des positions restantes
        if role == "Défenseur":
            role = "Milieu de terrain"
        elif role == "Milieu de terrain":
            role = "Attaquant"
        elif role == "Attaquant":
            role = "Défenseur"
        return get_position(role, role_count[role], role_positions, role_count)

# Fonction pour créer le graphique de passes
def visualize_pass_network(player_positions_df, passes_df):
    # Convertir les données de la dataframe en dictionnaires
    role_count = {"Défenseur": 0, "Milieu de terrain": 0, "Attaquant": 0, "Gardien": 0}
    players_positions = {}
    
    for index, row in player_positions_df.iterrows():
        role = row['role']
        position = get_position(role, role_count[role], role_positions, role_count)
        role_count[role] += 1
        players_positions[row['player.name']] = position

    # Calculer les informations des passes
    pass_info = passes_df.groupby(['player.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    pass_info_received = passes_df.groupby(['pass.recipient.name']).agg(
        num_passes_received=('pass.length', 'size'),
        avg_pass_length_received=('pass.length', 'mean')
    ).reset_index()

    # Créer des dictionnaires pour les infos de passes
    pass_info_dict = {
        row['player.name']: {
            'num_passes': row['num_passes'],
            'avg_pass_length': row['avg_pass_length']
        }
        for index, row in pass_info.iterrows()
    }

    pass_info_received_dict = {
        row['pass.recipient.name']: {
            'num_passes_received': row['num_passes_received'],
            'avg_pass_length_received': row['avg_pass_length_received']
        }
        for index, row in pass_info_received.iterrows()
    }

    # Tracer le terrain de football
    fig = go.Figure()

    # Ajouter des formes pour représenter le terrain de football
    terrain_shapes = [
        dict(type="rect", x0=0, y0=0, x1=120, y1=80, line=dict(color="Black")),
        dict(type="line", x0=60, y0=0, x1=60, y1=80, line=dict(color="Black", dash="dot")),
        dict(type="rect", x0=0, y0=18, x1=18, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=102, y0=18, x1=120, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=30, x1=5.5, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=114.5, y0=30, x1=120, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=36, x1=2, y1=44, line=dict(color="Black")),
        dict(type="rect", x0=118, y0=36, x1=120, y1=44, line=dict(color="Black")),
        dict(type="circle", xref="x", yref="y", x0=50.4, y0=30.4, x1=69.6, y1=49.6, line_color="Black"),
    ]
    fig.update_layout(shapes=terrain_shapes)

    # Ajouter les joueurs sur le terrain avec les infos de passes
    for player, pos in players_positions.items():
        passes_from_player = pass_info_dict.get(player, {'num_passes': 0, 'avg_pass_length': 0})
        passes_to_player = pass_info_received_dict.get(player, {'num_passes_received': 0, 'avg_pass_length_received': 0})

        # Calculer les passes entre ce joueur et les autres
        passes_to_other_players = passes_df[passes_df['player.name'] == player]
        passes_from_other_players = passes_df[passes_df['pass.recipient.name'] == player]

        passes_info_text = ""
        for idx, pass_row in passes_to_other_players.iterrows():
            recipient = pass_row['pass.recipient.name']
            if recipient in players_positions:
                passes_info_text += f"Passes vers {recipient}: {pass_row['pass.length']} m<br>"
        
        for idx, pass_row in passes_from_other_players.iterrows():
            sender = pass_row['player.name']
            if sender in players_positions:
                passes_info_text += f"Passes de {sender}: {pass_row['pass.length']} m<br>"

        hover_text = (
            f"Joueur: {player}<br>"
            f"Nombre de passes effectuées: {passes_from_player['num_passes']}<br>"
            f"Longueur moyenne des passes effectuées: {passes_from_player['avg_pass_length']:.2f} m<br>"
            f"Nombre de passes reçues: {passes_to_player['num_passes_received']}<br>"
            f"Longueur moyenne des passes reçues: {passes_to_player['avg_pass_length_received']:.2f} m<br>"
            f"{passes_info_text}"
        )

        fig.add_trace(go.Scatter(
            x=[pos[0]],
            y=[pos[1]],
            mode='markers+text',
            text=[player],
            textposition="top center",
            marker=dict(size=20, color='blue'),
            hovertext=hover_text,
            hoverinfo="text"
        ))

    # Calculer le nombre et la longueur moyenne des passes entre les joueurs
    passes_between_players = passes_df.groupby(['player.name', 'pass.recipient.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    # Ajouter les passes
    for row in passes_between_players.itertuples():
        if row[1] in players_positions and row[2] in players_positions:
            start = players_positions[row[1]]
            end = players_positions[row[2]]
            hover_text = (
                f"Passes de {row[1]} à {row[2]}<br>"
                f"Nombre de passes: {row.num_passes}<br>"
                f"Longueur moyenne des passes: {row.avg_pass_length:.2f} m"
            )
            fig.add_trace(go.Scatter(
                x=[start[0], end[0]],
                y=[start[1], end[1]],
                mode='lines',
                line=dict(width=2, color='red'),
                hovertext=hover_text,
                hoverinfo="text"
            ))

    fig.update_layout(
        title="Réseau de passes sur un terrain de football",
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False),
        width=1200,
        height=900
    )
    fig.show()
visualize_pass_network(df_positions, df_passes)

In [10]:
import pandas as pd
import plotly.graph_objs as go

# Exemple de données
data = {
    'Metrics': ['Passes completed', 'Forward pass%', 'Progressive passes', 'Defensive actions', 'Defensive duel%', 'Aerial%', 'Progressive carries'],
    'Pau Cubarsí 23/24': [96.2, 84.3, 63.7, 56.4, 90.1, 12.5, 65.7],
    'Nacho 23/24': [78.6, 97.8, 12.5, 5.0, 25.4, 16.4, 52.3]
}

df = pd.DataFrame(data)

# Fonction pour créer le radar chart
def create_radar_chart(df):
    fig = go.Figure()

    for column in df.columns[1:]:
        fig.add_trace(go.Scatterpolar(
            r=df[column],
            theta=df['Metrics'],
            fill='toself',
            name=column
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        showlegend=True,
        title="Comparaison des Performances des Joueurs"
    )

    fig.show()

create_radar_chart(df)


In [1]:
import dash
from dash import dcc, html
import dash_table
import pandas as pd
import plotly.graph_objs as go

# Exemple de données
data = {
    'Metrics': ['Passes completed', 'Forward pass%', 'Progressive passes', 'Defensive actions', 'Defensive duel%', 'Aerial%', 'Progressive carries'],
    'Pau Cubarsí 23/24': [96.2, 84.3, 63.7, 56.4, 90.1, 12.5, 65.7],
    'Nacho 23/24': [78.6, 97.8, 12.5, 5.0, 25.4, 16.4, 52.3]
}

df = pd.DataFrame(data)

# Tableau des informations supplémentaires
data_table = {
    'Player': ['Pau Cubarsí 23/24', 'Nacho 23/24'],
    'Team': ['Barcelona', 'Real Madrid'],
    'Age': [17, 34],
    'Passes completed': [96.2, 78.6],
    'Forward pass%': [84.3, 97.8],
    'Progressive passes': [63.7, 12.5],
    'Defensive actions': [56.4, 5.0],
    'Defensive duel%': [90.1, 25.4],
    'Aerial%': [12.5, 16.4],
    'Progressive carries': [65.7, 52.3],
    'Minutes played': [1707, 1940]
}

df_table = pd.DataFrame(data_table)

# Fonction pour créer le radar chart
def create_radar_chart(df):
    fig = go.Figure()

    for column in df.columns[1:]:
        fig.add_trace(go.Scatterpolar(
            r=df[column],
            theta=df['Metrics'],
            fill='toself',
            name=column
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        showlegend=True,
        title="Comparaison des Performances des Joueurs"
    )

    return fig

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Visualisation des Performances des Joueurs"),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df_table.columns],
        data=df_table.to_dict('records'),
        style_cell={'textAlign': 'center'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_as_list_view=True
    ),
    dcc.Graph(
        id='radar-chart',
        figure=create_radar_chart(df)
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


/tmp/ipykernel_4707/3552830408.py:3: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table
